In [ ]:
import scipy
import numpy as np
from matplotlib import pyplot as plt
from qiskit.circuit import QuantumCircuit, Parameter
from qiskit.circuit.library import PauliEvolutionGate
from qiskit.quantum_info import SparsePauliOp
from qiskit.synthesis import SuzukiTrotter, LieTrotter
from qiskit.transpiler.passes.synthesis.high_level_synthesis import HighLevelSynthesis, HLSConfig
from qiskit.transpiler.passes.synthesis.unitary_synthesis import UnitarySynthesis
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.transpiler import PassManager, CouplingMap
from qiskit.synthesis import MatrixExponential
from qiskit.quantum_info import Operator
from qiskit.visualization import array_to_latex
from qiskit import Aer, transpile
from qiskit_aer import AerSimulator
# from qiskit_ibm_runtime import QiskitRuntimeService, Estimator
from qiskit.primitives import Estimator, Sampler

In [ ]:
# example of exponentation of Pauli Z.
theta = Parameter(r"$\theta$")

qc_test = QuantumCircuit(4)
qc_test.cx(0,1)
qc_test.cx(1,2)
qc_test.cx(2,3)
qc_test.rz(theta, 3)
qc_test.cx(2,3)
qc_test.cx(1,2)
qc_test.cx(0,1)

qc_test.draw("mpl", filename="ladder")

In [ ]:
# Make 3-qubit Heisenberg Ham to test and corresponding PauliEvolutionGates.
test_time_evolve = 3
ham_terms = ["XXI", "YYI", "ZZI", "IXX", "IYY", "IZZ"]
# ham_terms = ["ZZ", "IX", "XI"] uncomment this for 2q tranverse XY model for the poster
test_heisenberg_ham = SparsePauliOp(data=ham_terms, 
                                    coeffs=np.ones(len(ham_terms), dtype=int))
test_paulievolutiongate = PauliEvolutionGate(operator=test_heisenberg_ham, 
                                             time=test_time_evolve,
                                             synthesis=MatrixExponential())


# Make initial circuit with PauliEvolutionGate.
qc_before = QuantumCircuit(len(ham_terms[0]))
qc_before.append(test_paulievolutiongate, range(len(ham_terms[0])))

qc_before.draw("mpl", filename="before.png")

In [ ]:
# Make cartan decomposed circuit with PauliEvolutionGates by calling the custom pass.
hls_config = HLSConfig(PauliEvolution=[("cartan", {"random_seed":4, "involution":"countY"})])
hls_pass = HighLevelSynthesis(hls_config)
qc_after = hls_pass(qc_before)

# hls_config = HLSConfig(PauliEvolution=[("cartan", {"random_seed":4, "involution":"countY"})])
# pm_hls = PassManager()
# pm_hls.append(HighLevelSynthesis(hls_config=hls_config))
# qc_hls = pm_hls.run(qc_before)

In [ ]:
display(qc_hls.draw("mpl", filename="after.png"))
qc_hls.count_ops()

# Time-Simulation of Heisenberg Model for $t \in \left[ 0, 2\pi \right]$ for initial state $| 0 0 0 \rangle$. Note the magnetic field in the X-direction.

Exact

In [ ]:
# Make observable for magnetization.
# observable_terms = ["ZII", "IZI", "IIZ"]
# observable = SparsePauliOp(data=observable_terms, coeffs=np.ones(len(observable_terms), dtype=int))
                           
# Simulation.
np.random.seed(1)
ham_terms = ["XXI", "YYI", "ZZI", "IXX", "IYY", "IZZ", "XII", "IXI", "IIX"]
coeffs_array = np.random.rand(len(ham_terms))
test_heisenberg_ham = SparsePauliOp(data=ham_terms, 
                                coeffs=coeffs_array)
t_param = Parameter("t_param")
# expval_array = []
prob_000_array_exact = []
t_array = np.linspace(start=0, stop=6*np.pi, num=1000)

for t in t_array:
    
    # Make PauliEvolutionGate
    test_paulievolutiongate = PauliEvolutionGate(operator=test_heisenberg_ham, 
                                             time=t,
                                             synthesis=MatrixExponential())
    
    # Make Circuit.
    qc_before = QuantumCircuit(3)
    # qc_before.initialize([0,1], 0)
    qc_before.append(test_paulievolutiongate, range(3))
    qc_before.measure_all()
    # display(qc_before.draw("mpl"))

    # Run circuit and get expval.
    # estimator = Estimator()
    # expval = estimator.run(qc_before, observable, shots=1024*(2**8)).result().values[0]

    # Append expval.
    #expval_array.append(expval)

    # Run circuit and get counts/num_shots.
    sampler = Sampler()
    job = sampler.run(qc_before) # shots=None means that sampler is retrieving from distribution directly rather than counts
    result = job.result()
    quasi_dict = result.quasi_dists[0]

    # Get the overlap with |000>. If 0 doesn't exist in distribution then can safely assume the |000> component is negligible.
    if 0 in quasi_dict:
        prob_000_array_exact.append(quasi_dict[0])
    else:
        prob_000_array_exact.append(0)

In [ ]:
Cartan. Note the choice of involution and seed.

In [ ]:
prob_000_array_cartan = []
t_param = Parameter("t_param")

# Make PauliEvolutionGate
test_paulievolutiongate = PauliEvolutionGate(operator=test_heisenberg_ham, 
                                            time=t_param,
                                            synthesis=MatrixExponential())

# Make Circuit.
qc_before = QuantumCircuit(3)
qc_before.append(test_paulievolutiongate, range(3))
qc_before.measure_all()

# Cartan decompose.
hls_config = HLSConfig(PauliEvolution=[("cartan", {"random_seed":-1, "involution":"countZ"})])
pm_hls = PassManager()
pm_hls.append(HighLevelSynthesis(hls_config=hls_config))
qc_cartan = pm_hls.run(qc_before) 

for t in t_array:
    
    # Run circuit and get counts/num_shots.
    sampler = Sampler()
    job = sampler.run(qc_cartan.bind_parameters({t_param: t})) # shots=None means that sampler is retrieving from distribution directly rather than counts
    result = job.result()
    quasi_dict = result.quasi_dists[0]

    # Get the overlap with |000>. If 0 doesn't exist in distribution then means the |000> component is likely
    if 0 in quasi_dict:
        prob_000_array_cartan.append(quasi_dict[0])
    else:
        prob_000_array_cartan.append(0)

Suzuki-Trotter

In [ ]:
# Make observable for magnetization.
# observable_terms = ["ZII", "IZI", "IIZ"]
# observable = SparsePauliOp(data=observable_terms, coeffs=np.ones(len(observable_terms), dtype=int))
                           
# Simulation.
t_param = Parameter("t_param")
# expval_array = []
prob_000_array_suzuki = []

for t in t_array:
    
    # Make PauliEvolutionGate
    test_paulievolutiongate = PauliEvolutionGate(operator=test_heisenberg_ham, 
                                                 time=t)
    
    # Make Circuit.
    qc_before = QuantumCircuit(3)
    # qc_before.initialize([0,1], 0)
    qc_before.append(SuzukiTrotter(order=2, reps=8).synthesize(test_paulievolutiongate), range(3))
    qc_before.measure_all()
    # display(qc_before.draw("mpl"))

    # Run circuit and get expval.
    # estimator = Estimator()
    # expval = estimator.run(qc_before, observable, shots=1024*(2**8)).result().values[0]

    # Append expval.
    #expval_array.append(expval)

    # Run circuit and get counts/num_shots.
    sampler = Sampler()
    job = sampler.run(qc_before) # shots=None means that sampler is retrieving from distribution directly rather than counts
    result = job.result()
    quasi_dict = result.quasi_dists[0]

    # Get the overlap with |000>.
    if 0 in quasi_dict:
        prob_000_array_suzuki.append(quasi_dict[0])
    else:
        prob_000_array_suzuki.append(0)

In [ ]:
# Plot simulation results.
%matplotlib inline
plt.figure(dpi=300)
plt.plot(t_array, prob_000_array_exact, label="Exact", color="blue")
plt.plot(t_array, prob_000_array_cartan, label="Cartan", color="orange", linestyle='dashed')
plt.plot(t_array, prob_000_array_suzuki, label="Suzki-Trotter", color="purple", linestyle='dotted')
plt.legend()
plt.xlabel(r"$t$")
plt.xticks(ticks=[0, 2*np.pi, 4*np.pi, 6*np.pi], labels=[r"$0$", r"$2\pi$", r"$4\pi$", r"$6\pi$"])
plt.ylabel(r"$| \langle 000 | \psi_t \rangle |^2$")
# plt.show()
plt.savefig("test")

In [ ]:
# Hamiltonian and Experiment Specifications.
np.random.seed(1)
ham_terms = ["XXI", "YYI", "ZZI", "IXX", "IYY", "IZZ", "XII", "IXI", "IIX"]
coeffs_array = np.random.rand(len(ham_terms))
test_heisenberg_ham = SparsePauliOp(data=ham_terms, 
                                    coeffs=coeffs_array)

prob_000_array_exact = []
t_array = np.linspace(start=0, stop=6*np.pi, num=1000)

# Time Evolution.
for t in t_array:

    # Run circuit and get counts/num_shots.
    sampler = Sampler()

    # Make PauliEvolutionGate
    test_paulievolutiongate = PauliEvolutionGate(operator=test_heisenberg_ham, 
                                                time=t,
                                                synthesis=MatrixExponential())

    # Make Circuit.
    qc_before = QuantumCircuit(3)
    qc_before.append(test_paulievolutiongate, range(3))
    qc_before.measure_all()

    # Run Circuit and get "quasi-probability distribution".
    job = sampler.run(qc_before) # shots=None means that sampler is retrieving from distribution directly rather than shot-based counts.
    result = job.result()
    quasi_dict = result.quasi_dists[0] # 0 <-> 000, 1 <-> 001, etc. (convert to binary)

    # Get the overlap with |000>. If 0 doesn't exist as a key in distribution dict then can safely assume the |000> component is negligible.
    if 0 in quasi_dict:
        prob_000_array_exact.append(quasi_dict.binary_probabilities["000"])
    else:
        prob_000_array_exact.append(0)

# Plot simulation results.
plt.figure(dpi=300)
plt.plot(t_array, prob_000_array_exact, label="Exact", color="blue")
plt.legend()
plt.xlabel(r"$t$")
plt.xticks(ticks=[0, 2*np.pi, 4*np.pi, 6*np.pi], labels=[r"$0$", r"$2\pi$", r"$4\pi$", r"$6\pi$"])
plt.ylabel(r"$| \langle 000 | \psi_t \rangle |^2$")
plt.show()